In [4]:
import sys

In [7]:
sys.path.append('/home/rustambaku13/Documents/Warwick/flink-streaming-gnn/python')

In [10]:
import jax
from flax.linen import relu, softmax, sigmoid
from nn.jax.multi_layer_dense import MultiLayerDense

In [11]:
predict_fn = MultiLayerDense(features=[16, 32, 7], activations=[relu, relu, softmax ])

In [12]:
predict_fn_params = predict_fn.init(jax.random.PRNGKey(0), jax.random.uniform(jax.random.PRNGKey(0), (7,)))

In [36]:
def loss(parameters, embedding, label):
    prediction = predict_fn.apply(parameters, embedding)
    label_probs = prediction * label + (1 - prediction) * (1 - label)
    return -jax.numpy.sum(jax.numpy.log(label_probs))
def batch_loss(parameters, batch_embeddings, batch_labels):
    losses = jax.vmap(loss, (None, 0, 0))(parameters, batch_embeddings, batch_labels)
    return jax.numpy.sum(losses)

In [29]:
labels = jax.numpy.eye(7)

In [30]:
logits = jax.random.uniform(jax.random.PRNGKey(0), (7,7))

In [37]:
batch_loss(predict_fn_params, logits, labels)

DeviceArray(20.200226, dtype=float32)

In [51]:
primals, f_vjp = jax.jvp(batch_loss, (predict_fn_params, logits, labels), )

TypeError: jvp() missing 1 required positional argument: 'tangents'

In [54]:
primals, f_vjp = jax.vjp(batch_loss, predict_fn_params, logits, labels)

In [57]:
primals

DeviceArray(20.200226, dtype=float32)

In [39]:
jax.grad(batch_loss)(predict_fn_params, logits, labels)

FrozenDict({
    params: {
        layers_0: {
            bias: DeviceArray([ 0.19430672, -0.30369353,  0.07354623,  0.        ,
                          0.03676981,  0.11104526, -0.00134995,  0.31701842,
                          0.        ,  0.30291268,  0.        , -0.05217026,
                         -0.2415508 , -0.00965077,  0.        , -0.05038666],            dtype=float32),
            kernel: DeviceArray([[ 2.17217598e-02, -5.03895693e-02,  4.03563157e-02,
                           0.00000000e+00,  5.62220775e-02,  1.03150241e-01,
                          -7.63198070e-04, -1.03521325e-01,  0.00000000e+00,
                           1.90008819e-01,  0.00000000e+00,  5.44620492e-02,
                           2.82845162e-02,  5.12389019e-02,  0.00000000e+00,
                          -4.98207286e-03],
                         [ 1.32059595e-02, -1.04973912e-01,  3.16388607e-02,
                           0.00000000e+00,  1.26859471e-01,  2.03986257e-01,
                    

In [58]:
a = jax.numpy.array([1,2,3])
b = jax.numpy.array([1,2,3])

In [59]:
a * b


DeviceArray([1, 4, 9], dtype=int32)